In [52]:
# 取得列表中每個工作的「職缺名稱」、「職缺連結」、「公司名稱」、「公司連結」、「地區」的資訊，dict 的 key 名稱自訂，而後將各別工作 dict 資料 append 在自訂的 list 當中，最後將 list 以 JSON 格式儲存。
# 一般來說，到第 15 頁就會停住，要求手動按下第 16 頁，之後每一頁都會要求手動按下，我們這裡不會要求做到自動按分頁，到第 15 頁含之前的職缺內容即可

In [1]:
# 選python 台北市/新北市 全職
# 滾到page 15卡住,p16在放click()
# 擷取職缺 公司名稱 地區那行


In [7]:
import re,time,requests,json,os
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from urllib import parse
from time import sleep
my_header={'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Mobile Safari/537.36'}
# my_params={
#     'ro':'1',
#     'expansionType':'area'',spec'',com'',job'',wf'',wktm',
#     'area':'6001001000'',6001002000',
#     'order':'11',
#     'asc':'0',
#     'page':'1',
#     'mode':'s',
#     'jobsource':'2018indexpoc',
    
#     }
# url=requests.get('https://www.104.com.tw/jobs/search/?',my_params,headers=my_header).url

options=webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver=webdriver.Chrome(options=options)
listData = []

def visit():
    driver.get('https://www.104.com.tw/jobs/main/')
def search():
    
    
    inputElement=driver.find_element(By.CSS_SELECTOR,"span#icity").click()
    sleep(2)
    inputElement=driver.find_element(By.CSS_SELECTOR,"input[value='6001001000']").click()
    sleep(0.5)
    inputElement=driver.find_element(By.CSS_SELECTOR,"input[value='6001002000']").click()
    sleep(0.5)
    inputElement=driver.find_element(By.CSS_SELECTOR,"button.category-picker-btn-primary").click()
    
    inputElement=driver.find_element(By.CSS_SELECTOR,"input#ikeyword")
    inputElement.send_keys("python")
    inputElement.click()
    sleep(0.6)
    cssSelectorbtn="btn btn-primary js-formCheck"
    btnInput=driver.find_element(By.CSS_SELECTOR,"button.btn.btn-primary.js-formCheck")
    btnInput.click()
    
    inputElement=driver.find_element(By.CSS_SELECTOR,"#js-job-tab > li:nth-child(2)").click()
def scroll():
    innerHeightOfWindow= 0
    totalOffset= 0
    while totalOffset<= innerHeightOfWindow:
        totalOffset += 90;
        js_scroll='''(
            function(){{
                window.scrollTo({{
                    top:{},
                    behavior:'smooth'
                }});
            }})();'''.format(totalOffset)
        driver.execute_script(js_scroll)
#         sleep(1)
        innerHeightOfWindow=driver.execute_script('return window.document.documentElement.scrollHeight;')
        if innerHeightOfWindow - totalOffset < 200:
            sleep(5)
            buttons = driver.find_elements(By.CSS_SELECTOR, "button.b-btn.b-btn--link.js-more-page")
            try:
                buttons[-1].click()
            except:
                continue
        if innerHeightOfWindow - totalOffset <= 100:
            sleep(5)
def parse():
    soup=bs(driver.page_source,'html.parser')
    url2='https://www.104.com.tw'
    jTitle=[]
    jLink=[]
    cTitle=[]
    cLink=[]
    bArea=[]
    Area=[]
    
    
#職缺名稱(jTitle)
    jobPage=driver.find_elements(By.CSS_SELECTOR,'article.b-block--top-bord.job-list-item.b-clearfix.js-job-item')
    for job in jobPage:
        jTitle=job.get_attribute('data-job-name')
        j=job.find_element(By.CSS_SELECTOR,"a.js-job-link")
        jLink=j.get_attribute('href')
        
        cTitle=job.get_attribute('data-cust-name')
#         c=job.find_element_by_xpath('//*[@id="js-job-content"]/article[2890]/div[1]/ul[1]/li[2]/a').send_keys("by_xpath")
        c=job.find_element(By.CSS_SELECTOR,'div.b-block__left > ul:nth-child(2) > li:nth-child(2) > a') 
        cLink=c.get_attribute('href')
        Area=job.find_element(By.CSS_SELECTOR,"ul.b-list-inline.b-clearfix.job-list-intro.b-content>li").text
        

                
        listData.append({

            "job_name": jTitle,
            "job_link": jLink,
            "com_name": cTitle,
            "com_link":cLink,
            "area":Area
        })
    
        
    
def saveJson():
    fp = open("104人力銀行3.json", "w", encoding='utf-8')
    fp.write( json.dumps(listData, ensure_ascii=False) )
    fp.close()
                
def close():
    driver.quit() 


if __name__=='__main__':
    visit()
    search()
    scroll()
    parse()
    saveJson()
    close()
    print('ok')
    

ok


In [13]:
# for i in range(20):
#     driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
#     time.sleep(0.6)
    

In [2]:
soup=bs(driver.page_source,'html.parser')
url2='https://www.104.com.tw'
jTitle=[]
jLink=[]
jLink2=[]
cTitle=[]
cLink=[]
bArea=[]
Area=[]
key=jLink2
#職缺名稱(jTitle)
job=soup.findAll('a',{'class':'dir'})
for a in job:
    jTitle.extend(a.find("h2", class_ = "title"))

#職缺連結(jLink)
job_link=soup.findAll('a',{'class':'dir'})
for a in job_link:
    if a .has_attr('href'):
        jLink.append(url2+a['href'])
        
#公司名稱(cTitle)
for b in job:
    cTitle.extend(b.find("h3",class_="company"))
#公司連結(cLink)
for  c in job_link:
    if c.has_attr('href'):
        jLink2.append(c['href'][5:9])
        cLink.append(url2+c['href'][0:10])
#地區(Area)
for d in job:
    bArea.extend(d.find("p"))
    for e in bArea:
        Area.append(e[0:6])



